Copyright (c) 2023, Douglas Santry
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, is permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.
2. Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR
ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES
(INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND
ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS
SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

In [1]:
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense

from math import sin as sine
from statistics import mean

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import pandas as pd


In [2]:
iris = pd.read_csv ("../Data/iris.csv")

In [3]:
dataset = iris.values                      # Extract the values from the dataframe into an array
x = dataset[:, 0:4].astype(float)          # Split off our input features


In [4]:
SpeciesToVector = OneHotEncoder (sparse=False)
TrainingLabels = SpeciesToVector.fit_transform (iris.loc[:, ["species"]])

y = TrainingLabels

In [5]:
iris_model = tf.keras.models.Sequential ()

iris_model.add (tf.keras.layers.Normalization (input_dim = 4))
iris_model.layers[0].adapt (x)

iris_model.add (Dense (20, input_shape=[1], activation='sigmoid'))
iris_model.add (Dense (20, activation='sigmoid'))
iris_model.add (Dense (3, activation='softmax'))

iris_model.build ()
iris_model.summary ()

Metal device set to: Apple M1


2023-12-21 12:57:25.797141: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-21 12:57:25.797225: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-12-21 12:57:25.859828: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-12-21 12:57:25.896614: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 4)                9         
 n)                                                              
                                                                 
 dense (Dense)               (None, 20)                100       
                                                                 
 dense_1 (Dense)             (None, 20)                420       
                                                                 
 dense_2 (Dense)             (None, 3)                 63        
                                                                 
Total params: 592
Trainable params: 583
Non-trainable params: 9
_________________________________________________________________


In [6]:
#
# Algorithm 5.6
#

ADAM = tf.keras.optimizers.Adam ()

In [7]:
#
# Algorithm 5.3, RPROP+
#

del_0 = 1e-2
del_min = 1e-8
del_max = 50

eta_plus = 1.2
eta_minus = 0.5

del_model = []
dL_model = []

for W in iris_model.trainable_variables:
    
    del_model.append (np.full (W.shape, del_0))
    dL_model.append (np.zeros (W.shape))


def RPROP (gradients):

    for i in range (len (iris_model.trainable_variables)):

        dL = gradients[i].numpy ()

        directions = np.sign (dL * dL_model[i])
        updates = np.sign (dL)

        case1 = directions > 0
        case2 = directions < 0
        case3 = directions == 0

        # Case 1
        del_model[i][case1] = np.minimum (del_model[i][case1] * eta_plus, del_max)
        updates[case1] = -updates[case1]

        # Case 2
        del_model[i][case2] = np.maximum (del_model[i][case2] * eta_minus, del_min)

        # Case 3
        updates[case3] = -updates[case3]

        # Apply the computed update and store new state
        iris_model.trainable_variables[i].assign_add (del_model[i] * updates)
        dL_model[i] = dL
        dL_model[i][case2] = 0.0

In [8]:
#
# Equation 3.40
#
# UpdateWeights in algorithm 3.3 (line 15 omitted as not required)
#

def UpdateWeights (grad):
    
    for i in range (len (grad)):
        iris_model.trainable_variables[i].assign_add (-0.01 * grad[i])


In [9]:
MLE = tf.keras.losses.CategoricalCrossentropy (from_logits=False, reduction=tf.keras.losses.Reduction.NONE)

In [10]:
#
# A training loop.  There are 3 optimizers to choose from.  Only uncomment 1 at time.
#

for epoch in range (500):
    
    with tf.GradientTape () as tape:
    
        # Note that we are not using predict.  This is a slower path that the tape
        # can monitor.
        y_ = iris_model (x, training=True)
    
        z = MLE (y_, y)
        Loss = tf.reduce_sum(z.numpy ()) / 150
    
    grad = tape.gradient (z, iris_model.trainable_variables)
    
    if epoch % 50 == 0:
        print ("Epoch %d: %f" % (epoch, Loss))

    if Loss < 0.11:
        break

#
# There are 3 possible, all mutually exclusive, update strategy choices.  Only uncomment one!
#

    RPROP (grad)
#    ADAM.apply_gradients (zip (grad, iris_model.trainable_variables))
#    UpdateWeights (grad)


Epoch 0: 10.595688
Epoch 50: 0.521850
Epoch 100: 0.227409
Epoch 150: 0.134707


In [11]:
y_ = iris_model.predict ([x], verbose=False)

sum (y_.argmax (1) == y.argmax (1)) / len (y)

2023-12-21 12:57:28.363505: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


0.9933333333333333

In [14]:
Loss.numpy ()

0.10998627845790467